In [1]:
import pandas as pd
from PIL import Image
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

import numpy as np

#Training the model now
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Create Training Data

In [2]:
train_data = pd.read_csv("sign_mnist_train.csv")
test_data = pd.read_csv("sign_mnist_test.csv")

def seperate_data(data):
    X = data.iloc[:, 1:].values
    Y = data.iloc[:, 0].values
    return X,Y

train_X, train_Y = seperate_data(train_data)
test_X, test_Y = seperate_data(test_data)


In [3]:
scaler = StandardScaler()

def encode(Y):
    encoder = OneHotEncoder()
    Y = Y.reshape(-1,1)
    Y = encoder.fit_transform(Y)
    Y = Y.toarray()
    return Y

train_Y = encode(train_Y)
test_Y = encode(test_Y)

train_X = scaler.fit_transform(train_X)
test_X = scaler.fit_transform(test_X)

print(test_Y.shape)


/home/abhishek/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


(7172, 24)


In [4]:
len(train_X)

27455

In [5]:
train_X = np.reshape(train_X, (len(train_X), 28, 28, 1))
test_X = np.reshape(test_X, (len(test_X), 28, 28, 1))


# Creating Model

In [6]:
classifier= Sequential()
classifier.add(Convolution2D(16, 3,3, input_shape=(train_X.shape[1:]), activation='relu'))
classifier.add(MaxPooling2D(pool_size =(2,2)))
#Flattening

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), input_shape=(28, 28, 1..., activation="relu")`
  


In [7]:

classifier.add(Flatten())
classifier.add(Dense(units=128, activation="relu"))
classifier.add(Dense(units=24, activation="sigmoid"))

classifier.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2704)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               346240    
_________________________________________________________________
dense_2 (Dense)              (None, 24)                3096      
Total params: 349,496
Trainable params: 349,496
Non-trainable params: 0
_________________________________________________________________


In [8]:
classifier.compile(optimizer="adam", loss="categorical_crossentropy", 
                   metrics=["accuracy"])


In [9]:
classifier.fit(train_X, train_Y, batch_size = 10, epochs = 2)


Epoch 1/2
27455/27455 [==============================] - 16s 599us/step - loss: 0.2581 - acc: 0.9269
Epoch 2/2
27455/27455 [==============================] - 16s 594us/step - loss: 0.0116 - acc: 0.9975


# Analyzing using test data

In [10]:
pred_Y = classifier.predict(test_X)
clean_pred_Y = []
for preds in pred_Y:
    index = 0
    max_val = preds[index]
    for i in range(len(preds)):
        if preds[i] > max_val:
            index = i
            max_val = preds[i]
    res = [0] * 24
    res[index] = 1
    clean_pred_Y.append(res)


In [16]:
correct = 0
for i in range(len(clean_pred_Y)):
    actual = test_Y[i].tolist()
    if clean_pred_Y[i].index(1) == actual.index(1.0):
        correct += 1
print("Total Correct: " + str(correct))
print("Correct/Total: " + str(correct))


Total Correct: 5532
Correct/Total: 5532


In [15]:
classifier.save('ASL_Model.h5')  # creates a HDF5 file 'my_model.h5'
print("Saved Model to ASL_Model.h5")

Saved Model to ASL_Model.h5
